# Mobile App for Discouraging Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions is looking to create an app that disinsentivizes lottery addicts by showing them the probability of their chances to win. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending their savings or taking out loans, accumulating debts, and eventually engage in desperate behaviors like theft. We are hoping that a more realistic view of how low the chances of winning the lottery actually are will discourage people from playing.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like: 

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

In [1]:
import pandas as pd

## Core Functions
Our goal is to write code that can enable users to answer probability questions about playing the lottery. So we'll need to calculate repeated probabilities. Let's start by writing two functions that we'll be using often: 

* <code>factorial()</code> - a function that calculates factorials
* <code>combinations()</code> - a function that calculates combinations

In [2]:
def factorial(n):
    total = 1
    for i in range(n,0,-1):
        total *= i
    
    return total

def combinations(n, k):
    c = factorial(n) / (factorial(k)*factorial(n-k))
    
    return c

## One Ticket Probability
For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket. So, we'll start by building a function that calculates the probability of winning for any given ticket.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins if the six numbers on their ticket matches all the six numbers drawn.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [3]:
# function determines the proability that the combination of 6 numbers the user has is a winning combination
def one_ticket_probability(n_list):
    n_combinations = combinations(49, 6)
    probability = 1 / n_combinations
    percentage = probability * 100
    
    print('''You have a {:.7f}% chance of winning the lottery with the chosen numbers. 
Your chances to win are 1 in {:,}'''.format(percentage, n_combinations))

In [4]:
# testing the function
n = [3, 45, 18, 9, 22, 13]
one_ticket_probability(n)

You have a 0.0000072% chance of winning the lottery with the chosen numbers. 
Your chances to win are 1 in 13,983,816.0


## Compare Ticket to Historical Data
We want to give users the opportunity to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now with that number combination. First we'll check out the historical data, then will build a function to do the comparison.
* Data can be downloaded: [here](https://www.kaggle.com/datasets/datascienceai/lottery-dataset)

In [5]:
# read in data
lottery = pd.read_csv('649.csv')
# quick glimpse of data
print(lottery.shape)
lottery.head()

(3665, 11)


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


The dataset contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. Each one of the 'NUMBER DRAWN' columns contains 1 of the 6 numbers for the ticket's combination.

Now we're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now. The engineering team told us that we need to be aware of the following details:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list and serve as an input to our function.
* The engineering team wants us to write a function that prints:
 - the number of times the combination selected occurred in the Canada data set; and
 - the probability of winning the big prize in the next drawing with that combination.

In [6]:
# extract 6 number combination that was drawn
def extract_numbers(row):
    c = {
            row['NUMBER DRAWN 1'],
            row['NUMBER DRAWN 2'],
            row['NUMBER DRAWN 3'],
            row['NUMBER DRAWN 4'],
            row['NUMBER DRAWN 5'],
            row['NUMBER DRAWN 6'],
    }
    return c
# check if the combination exists in historical data
def check_historical_occurrence(n_list, winning_ns):
    n_set = set(n_list)
    win_match = sum(winning_ns == n_set)
    n_combinations = combinations(49, 6)
    probaility_percentage = (1 / n_combinations) * 100
    
    if win_match == 0:
        print('''This combination has never occured.
This doesn't mean it's more likely to occur now. You have a {:.7f}% chance of winning in the next drawing with this combination. In other words, you have a 1 in 13,983,816 chances to win.'''.format(probaility_percentage))
    
    else:
        print('''The number of times this combination has been the winning combination is {}. 
You have a {:.7f}% chance of winning in the next drawing with this combination'''.format(win_match, probaility_percentage))

In [7]:
# testing functions
winning_ns = lottery.apply(extract_numbers, axis=1) # extract all winning combinations from historical data

n_list = [3, 11, 12, 14, 41, 44]
check_historical_occurrence(n_list, winning_ns)

This combination has never occured.
This doesn't mean it's more likely to occur now. You have a 0.0000072% chance of winning in the next drawing with this combination. In other words, you have a 1 in 13,983,816 chances to win.


## Probabilty for Multiple Tickets
Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. We're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [8]:
def multi_ticket_probability(n_tickets):
    n_combinations = combinations(49,6)
    percentage_probability = (n_tickets / n_combinations) * 100
    
    print('''With {} ticket(s), your chances of winning the big prize are {:6f}%'''.format(n_tickets, percentage_probability))

In [9]:
# testing function
multi_ticket_probability(6991908)

With 6991908 ticket(s), your chances of winning the big prize are 50.000000%


In [10]:
# testing function
multi_ticket_probability(5)

With 5 ticket(s), your chances of winning the big prize are 0.000036%


## Less Than 6 Winning Numbers
In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

* Inside the app, the user inputs:
 - six different numbers from 1 to 49; and
 - an integer between 2 and 5 that represents the number of winning numbers expected
* Our function prints information about the probability of having the inputted number of winning numbers.

The function below will calculate the probaility that a player's ticket has the amount of correct numbers specified by the user. If the user inputs 5, the function will output the probability that the ticket has 5 of the 6 numbers from the winning combination. 

In [11]:
def probability_less_6(n_winning_ns):
    combs = combinations(6, n_winning_ns)
    successful_outcomes = combs * 43
    total_outcomes = combinations(49, 6)
    probability_percentage = (successful_outcomes / total_outcomes) * 100
    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_ns, probability_percentage,
                                                               int(combs)))

In [12]:
# testing function
for test in [2,3,4,5]:
    probability_less_6(test)
    print('\n')

Your chances of having 2 winning numbers with this ticket are 0.004612%.
In other words, you have a 1 in 15 chances to win.


Your chances of having 3 winning numbers with this ticket are 0.006150%.
In other words, you have a 1 in 20 chances to win.


Your chances of having 4 winning numbers with this ticket are 0.004612%.
In other words, you have a 1 in 15 chances to win.


Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 6 chances to win.


